In [ ]:
import json
import openai
from dotenv import load_dotenv
import os
import re

# ===== 환경 설정 =====
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")
client = openai.Client()

# 태도/모범/성실 칭찬만 제거 (질문 출처로는 쓰지 않기 위함)
EXCLUDE_KEYWORDS = [
    "성실", "성실함", "성실하게", "모범", "모범적인",
    "태도", "태도가", "자세가 좋", "정성을 다해",
    "발표한 모습이 보기 좋았", "발표를 잘함", "발표를 잘 하",
    "참여도가 높", "열심히 참여", "적극적으로 참여",
    "친구들과 잘 지내", "봉사정신이 투철", "예의바른 태도"
]


##########################################################
# 1. 전체 텍스트 합치기
##########################################################
def get_full_text(student_data):
    """academic_records + reading을 하나의 큰 텍스트로 합침"""
    records = student_data.get("academic_records", [])
    if isinstance(records, list):
        full = "\n".join(str(x) for x in records)
    else:
        full = str(records)

    reading = student_data.get("reading", "")
    if reading:
        full += "\n" + str(reading)

    return full


##########################################################
# 2. 희망분야 → 학년별 자동 배정 (1,2,3학년 순서대로)
##########################################################
def extract_career_by_grade(full_text):
    """
    '희망 분야 ~~~' 가 등장하는 순서대로
    1학년, 2학년, 3학년에 매칭한다고 가정.
    """
    matches = re.findall(r"희망\s*분야\s*([^\n]+)", full_text)

    grade_raw = {1: None, 2: None, 3: None}
    for idx, raw in enumerate(matches[:3], start=1):
        cleaned = raw.replace("분야", "").replace("계열", "").strip()
        grade_raw[idx] = cleaned

    return grade_raw


##########################################################
# 3. 세특/창체 출처 자동 추출
##########################################################
def extract_sources(full_text):
    """
    - '과목명: 내용...' 형태를 세특으로 가정
    - '동아리활동', '자율활동', '진로활동', '봉사활동' 블록은 창체로 처리
    """
    sources = []

    # 3-1) 세부능력특기사항
    pattern = re.compile(
        r"([가-힣A-Za-z0-9\s]+):\s*(.+?)(?=\n[가-힣A-Za-z0-9\s]+:|\n동아리활동|\n자율활동|\n진로활동|\n봉사활동|\Z)",
        re.DOTALL
    )

    for m in pattern.finditer(full_text):
        subject = m.group(1).strip()
        desc = m.group(2).strip().replace("\n", " ")
        label = f"{subject}(세부능력특기사항)"
        sources.append((label, desc))

    # 3-2) 창체
    blocks = ["동아리활동", "자율활동", "진로활동", "봉사활동"]
    for b in blocks:
        bpat = re.compile(
            b + r"\s*\n(.+?)(?=\n동아리활동|\n자율활동|\n진로활동|\n봉사활동|\Z)",
            re.DOTALL
        )
        for m in bpat.finditer(full_text):
            desc = m.group(1).strip().replace("\n", " ")
            label = b  # 예: "진로활동"
            sources.append((label, desc))

    return sources


##########################################################
# 4. 태도성 내용 제거
##########################################################
def filter_out_attitude(sources):
    """태도/성실 칭찬 위주의 기록은 질문 출처에서 제외"""
    clean = []
    for label, text in sources:
        combo = label + " " + text
        if not any(bad in combo for bad in EXCLUDE_KEYWORDS):
            clean.append((label, text))
    return clean


##########################################################
# 5. OpenAI로 질문 50개 생성 (유형 분류 포함)
##########################################################
def generate_questions(full_text, career_by_grade, sources):
    """
    - 전공 적합성 / 활동·경험 기반 / 인성·가치관 유형으로
      섞어서 질문 50개 생성
    - 비율은 고정하지 않고, 생기부상 '중요해 보이는 포인트'를 더 많이 묻는다.
    """

    # 생기부가 너무 길면 일부만 잘라서 전달 (토큰 절약용)
    truncated_text = full_text[:6000]

    # 출처도 너무 많으면 상위 몇 개만
    max_sources = 30
    brief_sources = sources[:max_sources]

    sources_str = ""
    for i, (label, desc) in enumerate(brief_sources, start=1):
        sources_str += f"{i}. 출처: {label}\n   내용: {desc[:200]}\n"

    career_str = ""
    for g in [1, 2, 3]:
        career_str += f"{g}학년 희망분야: {career_by_grade.get(g)}\n"

    # 🔥 여기 system_prompt를 대폭 수정 (비율 고정 X, 중요 포인트 위주)
    system_prompt = """
당신은 대한민국 대학 입학 면접관입니다.
주어진 학생 생기부 내용을 바탕으로, 다음 세 가지 유형으로 인터뷰 질문을 만드세요.

- 전공 적합성
- 활동·경험 기반
- 인성·가치관

하지만, 각 유형의 개수를 억지로 균등하게 맞추지 마십시오.
이 학생을 평가할 때 "정말로 중요한 포인트"를 중심으로 질문 비중을 조절하십시오.

우선순위 예시:
1) 학년별 희망분야와 직접적으로 연결되는 과목/세특/진로활동
2) 한 번 이상 반복해서 등장하거나, 분량이 길고 구체적인 활동
3) 동아리·프로젝트·탐구 활동 등에서 학생이 주도적으로 역할을 수행한 부분
4) 협업, 갈등 해결, 책임감, 선택의 근거 등이 드러나는 장면 (인성·가치관)

규칙:
1) 질문은 총 50개를 생성합니다.
2) 각 질문은 반드시 아래 세 유형 중 하나로 분류합니다.
   - 전공 적합성
   - 활동·경험 기반
   - 인성·가치관
3) 각 질문마다 '출처'를 명시합니다.
   - 예: '2학년 인공지능 수학(세부능력특기사항)', '3학년 진로활동', '1학년 동아리활동' 등
   - 출처는 아래 제공된 sources 목록을 참고해 작성하세요.
4) 학생에게 '답변하라'고 요구하지 말고, 질문만 작성합니다.
5) 질문은 가능한 한 구체적으로, 실제 면접에서 바로 사용할 수 있도록 작성합니다.
6) 전공 적합성 질문은 '희망분야 + 관련 과목/활동'에 가장 강하게 연결된 부분 위주로 만듭니다.
7) 활동·경험 기반 질문은 실제 세특/창체에서 특정 활동을 콕 집어, 계기·역할·어려움·결과를 캐묻는 방향으로 설계합니다.
8) 인성·가치관 질문은 학생의 선택 기준, 책임감, 협업 태도, 갈등 해결 방식이 드러나도록 설계합니다.
9) 질문 수의 비율은 중요하지 않습니다. 다만, 생기부에서 빈약한 부분보다는, 진로·전공과 강하게 연결된 부분과 깊게 써 있는 활동을 우선적으로 많이 질문하십시오.

출력 형식(정확히 지키세요):

1번
- 유형: 전공 적합성 / 활동·경험 기반 / 인성·가치관 중 하나
- 출처: (몇 학년, 어떤 과목 또는 어떤 활동에서 나온 질문인지)
- 질문: ~~~?

2번
- 유형: ...
- 출처: ...
- 질문: ...?

...

50번
- 유형: ...
- 출처: ...
- 질문: ...?
"""

    user_prompt = f"""
[학생 생기부 통합 텍스트 일부]
{truncated_text}

[학년별 희망분야]
{career_str}

[질문 설계에 참고할 주요 출처 목록]
{sources_str}

위 정보를 바탕으로, 총 50개의 면접 질문을 출력하세요.
"""

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        max_tokens=4000
    )

    return response.choices[0].message.content


##########################################################
# 6. 유형별 질문 개수 세기
##########################################################
def count_question_types(questions_block: str):
    """
    생성된 질문 블록 텍스트에서
    '- 유형: ...' 줄을 찾아 유형별 개수를 센다.
    """
    type_counts = {
        "전공 적합성": 0,
        "활동·경험 기반": 0,
        "인성·가치관": 0,
        "기타": 0
    }

    matches = re.findall(r"-\s*유형:\s*([^\n]+)", questions_block)

    for t in matches:
        t_clean = t.strip()
        if "전공" in t_clean:
            type_counts["전공 적합성"] += 1
        elif "활동" in t_clean or "경험" in t_clean:
            type_counts["활동·경험 기반"] += 1
        elif "인성" in t_clean or "가치관" in t_clean:
            type_counts["인성·가치관"] += 1
        else:
            type_counts["기타"] += 1

    return type_counts


##########################################################
# 7. 메인 실행
##########################################################
def main():
    with open("wnskadud_structured (1).json", "r", encoding="utf-8") as f:
        data = json.load(f)

    full_text = get_full_text(data)
    career_by_grade = extract_career_by_grade(full_text)
    sources = extract_sources(full_text)
    sources = filter_out_attitude(sources)

    print("=== 희망분야 인식 결과 ===")
    for g in [1, 2, 3]:
        print(f"{g}학년 → {career_by_grade.get(g)}")
    print("=========================\n")

    if not sources:
        print("⚠ 질문 출처로 사용할 기록이 없습니다. JSON 내용을 확인하세요.")
        return

    questions_block = generate_questions(full_text, career_by_grade, sources)

    print("\n===== 생성된 면접 질문 50개 =====\n")
    print(questions_block)
    print("\n================================\n")

    # 유형별 개수 세기
    type_counts = count_question_types(questions_block)

    print("=== 유형별 질문 개수 ===")
    print(f"- 전공 적합성: {type_counts['전공 적합성']}개")
    print(f"- 활동·경험 기반: {type_counts['활동·경험 기반']}개")
    print(f"- 인성·가치관: {type_counts['인성·가치관']}개")
    print(f"- 기타(분류 불명): {type_counts['기타']}개")
    print("========================")


if __name__ == "__main__":
    main()